In [1]:
# python notebook for Make Your Own Neural Network
# code for a 3-layer neural network, and code for learning the MNIST dataset
# (c) Tariq Rashid, 2016
# license is GPLv2

In [2]:
import numpy                # 행렬 계산을 위한 라이브러리
import scipy.special        # 시그모이드 함수를 위한 라이브러리
import matplotlib.pyplot    # 그림을 그리기 위한 라이브러리
#%matplotlib inline           # 주피터 노트북 내에서 그림 출력

In [3]:
# 신경망 클래스 정의
class neuralNetwork:
    
    
    # 신경망 초기화 함수
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # 입력 노드의 수, 은닉 노드의 수, 출력 노드의 수를 전달받아 변수에 저장
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # 가중치 행렬 wih와 who를 초기화
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))     # 입력 계층에서 은닉 계층으로 연결된 가중치 행렬을 정규화된 랜덤 숫자로 초기화
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))     # 은닉 계층에서 출력 계층으로 연결된 가중치 행렬을 정규화된 랜덤 숫자로 초기화

        # 학습률을 전달받아 저장
        self.lr = learningrate
        
        # activation_function이란 이름의 함수를 선언(활성화 함수 = 시그모이드 함수)
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # 신경망을 학습시키는 함수
    def train(self, inputs_list, targets_list):
        # inputs_list(입력 값)를 2차원 행렬로 변환하고 전치
        inputs = numpy.array(inputs_list, ndmin=2).T
        # targets_list(목표 값)를 2차원 행렬로 변환하고 전치
        targets = numpy.array(targets_list, ndmin=2).T
        
        # 은닉 계층에 입력되는 신호 값을 계산(행렬 곱)
        hidden_inputs = numpy.dot(self.wih, inputs)
        # 은닉 계층에서 출력되는 신호 값을 계산(활성화 함수) 
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # 최종 출력 계층에 입력되는 신호 값을 계산(행렬 곱)
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # 최종 출력 계층에서 출력되는 신호 값을 계산(활성화 함수)
        final_outputs = self.activation_function(final_inputs)
        
        # 출력 계층의 오차(목표 값 - 실제 계산된 값)
        output_errors = targets - final_outputs
        # 은닉 계층의 오차(출력 계층의 오차를 가중치에 따라 나누고 은닉 노드에서 재조합)
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # 은닉 계층과 출력 계층 사이의 오차를 적용하여 가중치 업데이트
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # 입력 계층과 은닉 계층 사이의 오차를 적용하여 가중치 업데이트
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # 신경망에 질의하는 함수
    def query(self, inputs_list):
        # inputs_list(입력 값)를 2차원 행렬로 변환하고 전치
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # 은닉 계층에 입력되는 신호 값을 계산(행렬 곱)
        hidden_inputs = numpy.dot(self.wih, inputs)
        # 은닉 계층에서 출력되는 신호 값을 계산(활성화 함수) 
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # 최종 출력 계층에 입력되는 신호 값을 계산(행렬 곱)
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # 최종 출력 계층에서 출력되는 신호 값을 계산(활성화 함수)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [4]:
# 입력 노드의 수, 은닉 노드의 수, 출력 노드의 수를 설정
input_nodes = 784       # MNIST 숫자 데이터의 픽셀 수(28*28)
hidden_nodes = 200      # 적절한 갯수
output_nodes = 10       # 찾아야 할 숫자는 10가지

# 학습률 설정
learning_rate = 0.1

# 신경망 인스턴스(객체) 생성
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [5]:
# 학습용 데이터 파일을 열어 그 내용을 리스트로 저장함
training_data_file = open("mnist_dataset/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [6]:
# 신경망을 학습시킴

# 학습을 반복할 횟수를 설정
epochs = 5

for e in range(epochs):
    # 학습용 데이터 파일에 있는 모든 레코드를 차례대로 학습시킴
    for record in training_data_list:
        # 컴마(,)를 기준으로 레코드를 잘라서 리스트로 저장
        all_values = record.split(',')
        # 레코드 숫자 값 범위(0~255)를 0.01~1.00의 범위로 변환
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # 목표 값 행렬을 생성하고 모든 값을 0.01로 초기화
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0]은 학습을 위한 답이 들어있으므로 해당 노드의 목표 값을 0.99로 설정
        targets[int(all_values[0])] = 0.99
        # 입력 데이터 값과 목표 값을 주고 학습시킴
        n.train(inputs, targets)
        pass
    pass

In [7]:
# 테스트용 데이터 파일을 열어 그 내용을 리스트로 저장함
test_data_file = open("mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [8]:
# 신경망에 질의하여 테스트

# 신경망 점수표를 빈 리스트로 생성
scorecard = []

# go through all the records in the test data set
# 테스트용 데이터 파일에 있는 모든 레코드를 차례대로 질의하여 테스트함
for record in test_data_list:
    # 컴마(,)를 기준으로 레코드를 잘라서 리스트로 저장
    all_values = record.split(',')
    # correct answer is first value
    # all_values[0]은 테스트를 위한 정답이 들어있음
    correct_label = int(all_values[0])
    # 레코드 숫자 값 범위(0~255)를 0.01~1.00의 범위로 변환
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # 신경망에 테스트 값을 주어 질의
    outputs = n.query(inputs)
    # 출력 노드 값 중 가장 큰 값이 신경망의 답
    label = numpy.argmax(outputs)
    # 신경망 점수표에 점수를 기록
    if (label == correct_label):
        # 신경망이 정답을 말하면 점수표에 1을 추가
        scorecard.append(1)
    else:
        # 신경망이 오답을 말하면 점수표에 0을 추가
        scorecard.append(0)
        pass
    
    pass

In [9]:
# 신경망의 성능(정답률)을 계산하여 출력
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.9712
